불/연기 , 연기 데이터셋 병합

연기 데이터셋 라벨을
불/연기 데이터셋에 맞게 변환

In [1]:
import os
import glob

root = "SmokeDetection_copy"  # 경로는 상황에 맞게 바꿔주세요
splits = ['train', 'valid', 'test']

for split in splits:
    label_dir = os.path.join(root, split, "labels")
    label_files = glob.glob(os.path.join(label_dir, "*.txt"))

    for path in label_files:
        with open(path, "r") as f:
            lines = f.readlines()

        updated_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == '0':
                parts[0] = '1'  # smoke → class 1
            updated_lines.append(" ".join(parts))

        with open(path, "w") as f:
            f.write("\n".join(updated_lines))


In [2]:
import os
import shutil

def merge_dataset(source, target, splits=['train', 'valid', 'test']):
    for split in splits:
        for subfolder in ['images', 'labels']:
            src_dir = os.path.join(source, split, subfolder)
            tgt_dir = os.path.join(target, subfolder, split)

            os.makedirs(tgt_dir, exist_ok=True)

            for file in os.listdir(src_dir):
                src_file = os.path.join(src_dir, file)
                tgt_file = os.path.join(tgt_dir, file)
                shutil.copy(src_file, tgt_file)

# 사용 예시
merge_dataset("FireSmokeDetection-4", "MergedDataset")
merge_dataset("SmokeDetection", "MergedDataset")


병합 데이터셋으로 학습시키기

In [6]:
from ultralytics import YOLO

# 1. 모델 로드
model = YOLO("yolov8n.pt")

# 2. 학습 시작
model.train(
    data="MergedDataset/data.yaml",
    epochs=120,
    imgsz = 768, # 이미지 크기 증가 (연기 표현 향상 예상)
    patience=15, # EarlyStoping 살짝 여유 있게
    batch=16, # 메모리에 따라 조절 (CPU=4~8, GPU=16~32)
    lr0=0.001, # 초기 학습률 낮게
    optimizer='AdamW', # 안정적 수렴
    dropout=0.1, # 경량 모델에서 과적합 방지
    warmup_epochs=3, # 초기 안정화
    device=0,
    project="runs", # 결과 저장 경로
    name="merge_detect_v8n", # 실험 이름
    resume=False # 중단된 학습 이어서 할 건지 여부
)

New https://pypi.org/project/ultralytics/8.3.154 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=MergedDataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=merge_detect_v8n4, nbs=64, nms=F

train: Scanning C:\Users\main\workspace\vision\project\MergedDataset\labels\train... 5222 images, 23 backgrounds, 0 corrupt: 100%|██████████| 5222/5222 [00:08<00:00, 619.43it/s]


train: New cache created: C:\Users\main\workspace\vision\project\MergedDataset\labels\train.cache
WARNING Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 10089. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
val: Fast image access  (ping: 0.10.0 ms, read: 4.70.9 MB/s, size: 42.6 KB)


val: Scanning C:\Users\main\workspace\vision\project\MergedDataset\labels\valid... 1157 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1157/1157 [00:02<00:00, 482.10it/s]

val: New cache created: C:\Users\main\workspace\vision\project\MergedDataset\labels\valid.cache


...

                   all       1157       2033       0.49      0.355      0.354      0.183
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 51, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



66 epochs completed in 0.947 hours.
Optimizer stripped from runs\merge_detect_v8n4\weights\last.pt, 6.3MB
Optimizer stripped from runs\merge_detect_v8n4\weights\best.pt, 6.3MB

Validating runs\merge_detect_v8n4\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:06<00:00,  5.62it/s]


                   all       1157       2033      0.488      0.384      0.382      0.192
                  fire        598       1219       0.56      0.559      0.553      0.298
                 smoke        728        814      0.416      0.209      0.211      0.086
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\merge_detect_v8n4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019F845DFF20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [8]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 시작
model.train(
    data="MergedDataset/data.yaml",
    epochs=150,
    imgsz=768,
    batch=16,
    lr0=0.0007,
    optimizer="AdamW",
    dropout=0.1,
    weight_decay=0.0005,
    warmup_epochs=5,
    patience=20,
    device=0,
    project="runs",
    name="merge_detect_v8s",
)


New https://pypi.org/project/ultralytics/8.3.154 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=MergedDataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0007, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=merge_detect_v8s2, nbs=64, nms=

train: Scanning C:\Users\main\workspace\vision\project\MergedDataset\labels\train.cache... 5222 images, 23 backgrounds, 0 corrupt: 100%|██████████| 5222/5222 [00:00<?, ?it/s]

WARNING Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 10089. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Fast image access  (ping: 0.10.0 ms, read: 163.444.6 MB/s, size: 42.6 KB)


val: Scanning C:\Users\main\workspace\vision\project\MergedDataset\labels\valid.cache... 1157 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1157/1157 [00:00<?, ?it/s]


...

                   all       1157       2033      0.528      0.367      0.355      0.193
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 63, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



83 epochs completed in 1.960 hours.
Optimizer stripped from runs\merge_detect_v8s2\weights\last.pt, 22.5MB
Optimizer stripped from runs\merge_detect_v8s2\weights\best.pt, 22.5MB

Validating runs\merge_detect_v8s2\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:10<00:00,  3.59it/s]


                   all       1157       2033      0.555      0.395      0.406      0.207
                  fire        598       1219      0.623      0.582      0.578      0.315
                 smoke        728        814      0.488      0.208      0.233     0.0985
Speed: 0.4ms preprocess, 3.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\merge_detect_v8s2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001A013DC8980>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

연기만

In [10]:
from ultralytics import YOLO

# 1. 모델 로드
model = YOLO("yolov8n.pt")

# 2. 학습 시작
model.train(
    data="SmokeDetection/data.yaml",
    epochs=120,
    imgsz = 768, # 이미지 크기 증가 (연기 표현 향상 예상)
    patience=15, # EarlyStoping 살짝 여유 있게
    batch=16, # 메모리에 따라 조절 (CPU=4~8, GPU=16~32)
    lr0=0.001, # 초기 학습률 낮게
    optimizer='AdamW', # 안정적 수렴
    dropout=0.1, # 경량 모델에서 과적합 방지
    warmup_epochs=3, # 초기 안정화
    device=0,
    project="runs", # 결과 저장 경로
    name="smoke_detect_v8n", # 실험 이름
    resume=False # 중단된 학습 이어서 할 건지 여부
)

New https://pypi.org/project/ultralytics/8.3.154 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=SmokeDetection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=smoke_detect_v8n2, nbs=64, nms=

train: Scanning C:\Users\main\workspace\vision\project\SmokeDetection\train\labels... 464 images, 0 backgrounds, 0 corrupt: 100%|██████████| 464/464 [00:00<00:00, 1354.05it/s]

train: New cache created: C:\Users\main\workspace\vision\project\SmokeDetection\train\labels.cache


val: Fast image access  (ping: 0.10.0 ms, read: 139.541.0 MB/s, size: 39.0 KB)


val: Scanning C:\Users\main\workspace\vision\project\SmokeDetection\valid\labels... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<00:00, 915.22it/s]

...

                   all         70         96      0.787      0.552      0.597      0.328
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



64 epochs completed in 0.084 hours.
Optimizer stripped from runs\smoke_detect_v8n2\weights\last.pt, 6.3MB
Optimizer stripped from runs\smoke_detect_v8n2\weights\best.pt, 6.3MB

Validating runs\smoke_detect_v8n2\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.97it/s]


                   all         70         96      0.842      0.594      0.667      0.384
Speed: 0.9ms preprocess, 1.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\smoke_detect_v8n2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019F9011AD80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [11]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("yolov8s.pt")

# 학습 시작
model.train(
    data="SmokeDetection/data.yaml",
    epochs=150,
    imgsz=768,
    batch=16,
    lr0=0.0007,
    optimizer="AdamW",
    dropout=0.1,
    weight_decay=0.0005,
    warmup_epochs=5,
    patience=20,
    device=0,
    project="runs",
    name="smoke_detect_v8s",
)


New https://pypi.org/project/ultralytics/8.3.154 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=SmokeDetection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0007, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=smoke_detect_v8s, nbs=64, nms=

train: Scanning C:\Users\main\workspace\vision\project\SmokeDetection\train\labels.cache... 464 images, 0 backgrounds, 0 corrupt: 100%|██████████| 464/464 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 465.1135.4 MB/s, size: 39.0 KB)


val: Scanning C:\Users\main\workspace\vision\project\SmokeDetection\valid\labels.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]


Plotting labels to runs\smoke_detect_v8s\labels.jpg... 
optimizer: AdamW(lr=0.0007, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 768 train, 768 val
Using 8 dataloader workers
Logging results to runs\smoke_detect_v8s
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         70         96      0.646      0.573      0.602      0.319
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 42, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



62 epochs completed in 0.139 hours.
Optimizer stripped from runs\smoke_detect_v8s\weights\last.pt, 22.5MB
Optimizer stripped from runs\smoke_detect_v8s\weights\best.pt, 22.5MB

Validating runs\smoke_detect_v8s\weights\best.pt...
Ultralytics 8.3.146  Python-3.12.9 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.74it/s]


                   all         70         96      0.741      0.562      0.637       0.38
Speed: 0.4ms preprocess, 4.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\smoke_detect_v8s


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019F490EA390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480